In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from shapely.geometry import Point
import geopandas as gpd

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# San Diego Region Crime Incidents 2007 - 2013
`sandag.gov-crime-2007e2013-2.1.1` Last Update: 2020-11-23T04:44:34

_Processed crime incidents for San Diego county, 2007-2013. Of our crime datasets, this one is most useful for analysis._

Processed crime incidents, based on data supplied by SANDAG.

Important:

  * See Caveats for limitations and warnings regarding this data. 
  * Use of this data is subject to multiple terms and conditions. See Terms for details. 

This dataset includes geocoded crime incidents from 1 Jan 2007 to 31 March 2013 that were returned by SANDAG for Public Records request 12-075. 


Asr\_zone Codes
===============

The integer values in the asr\_zone field are taken directly from the [SANGIS parcel data](http://rdw.sandag.org/file_store/Parcel/Parcels.pdf). These values are: 

	╔════╦═════════════════════════════════╗
	║ -1 ║ Unset                           ║
	║  0 ║ Unzoned                         ║
	║  1 ║ Single family residential (R-1) ║
	║  2 ║ Minor multiple (R-2)            ║
	║  3 ║ Restricted multiple (R-3)       ║
	║  4 ║ Multiple residential (R-4)      ║
	║  5 ║ Restricted commercial           ║
	║  6 ║ Commercial                      ║
	║  7 ║ Industrial (M zone)             ║
	║  8 ║ Agricultural                    ║
	║  9 ║ Special and/or misc.            ║
	╚════╩═════════════════════════════════╝				



Addresses and Geocoding
=======================
SANDAG returns the position of incidents as a block address, and occasionally as an intersection. Block addresses are the original address of the incident, with the last two digits set to '00'. 

Before geocoding, all of the original block addresses are normalized to be more consistent and to remove different versions of the same address. There are a few transformations performed on the address, including:

  * Converting street types synonyms like 'Avenue', 'Avenu' and 'ave.' to standard abbreviations like 'ave.'
  * Converting street directions ( 'West main Street' ) to abbreviations like 'W Main st'

Many geocoders are designed to work with mailable addresses, and block addresses are not real postal addresses. This data is geocoded with custom code that uses the SANGIS streets database, matching the block addresses to a street segment. This produces more sensible results, because the crime is attributed to an entire block, rather than to an arbitrary point on the block. However, with the crime is represented as a point, it will appear at the location of the center of the street segment, usually in the middle of the block. 

This means that all of the incidents on a block will appear at a single location. In most GIS programs, it is difficult to see that there are actually many points in one place. Be aware that each point you see may actually be dozens of incidents. 

The files that SANDAG returned included 1,008,524 incident records, and 953,824 records were geocoded (95%). The 'gctype' field has a value of NONE when the  record was not geocoded, and any field that depends on a locations, such as x, y, lon, lat, segment_id, community, and others, will have default values. 

Caveats
=======

As with most crime data, there are many issues, limitations and problems that users must be aware of to avoid making incorrect conclusions. 

*Crime incident data is inherently problematic.* Crime incident reports are collected by busy officers in stressful situations who are trying to describe complex situations with rigid categories. Virtually every point of the data collection process has multiple opportunities for errors and few opportunities for correction after the fact. Analysts must consider the difficulties of collecting crime data when assessing the validity of any conclusions. 

*Data is collected by 19 different agencies.* While the data is all sourced from SANDAG, it originates with 19 different police departments. These departments may have different policies that can result in different categorizations for the same crime, and they m

In [3]:
incidents = pd.concat([ r.dataframe() for r in pkg.references() ])


In [4]:

incidents = incidents.drop(columns=['id'])

In [5]:
incidents['date'] = pd.to_datetime(incidents.date.astype(str)+'T'+incidents.time.astype(str))
incidents['day'] = incidents.date.dt.day
incidents['week'] = incidents.date.dt.isocalendar().week
incidents['dow'] =  incidents.date.dt.weekday

geometry = [Point(xy) for xy in zip(incidents['lon'], incidents['lat'])]# Coordinate reference system : WGS84

incidents = gpd.GeoDataFrame(incidents, crs=4326, geometry=geometry)


In [11]:
import rowgenerators as rg

# Remap the old CPC codes back to names

city_cpc = rg.dataframe('gs+https://docs.google.com/spreadsheets/d/1tvXZKjyh5tdKHF2TUbQvW0hJoSjoG6dK4pA2SD-2pu4/edit#gid=0')
city_map = {r.code:r['name'] for idx, r in city_cpc.iterrows()}


com_cpc = rg.dataframe('gs+https://docs.google.com/spreadsheets/d/1tvXZKjyh5tdKHF2TUbQvW0hJoSjoG6dK4pA2SD-2pu4/edit#gid=1')
com_map = {r.code:r['name'] for idx, r in com_cpc.iterrows()}

ngh_cpc = rg.dataframe('gs+https://docs.google.com/spreadsheets/d/1tvXZKjyh5tdKHF2TUbQvW0hJoSjoG6dK4pA2SD-2pu4/edit#gid=2')
ngh_map = {r.code:r['name'] for idx, r in ngh_cpc.iterrows()}


incidents['city'] = incidents.city.replace(city_map)
incidents['nbrhood'] = incidents.nbrhood.replace(ngh_map)
incidents['community'] = incidents.community.replace(com_map)
incidents.rename(columns={'date':'datetime'}, inplace=True)
incidents = incidents.sort_values(['datetime','city','segment_id'])

incidents.head().T

,51,0,52,57,56
datetime,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00
year,2007,2007,2007,2007,2007
month,1,1,1,1,1
day,1,1,1,1,1
week,1,1,1,1,1
dow,0,0,0,0,0
time,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00
hour,0,0,0,0,0
is_night,1,1,1,1,1
type,THEFT/LARCENY,THEFT/LARCENY,THEFT/LARCENY,FRAUD,FRAUD


In [12]:
incidents[incidents.hour == 10]

,datetime,year,month,day,week,dow,time,hour,is_night,type,...,council,council_pop,asr_zone,lampdist,lat,lon,desc,gctype,gcquality,geometry
537,2007-01-01 10:00:00,2007,1,1,1,0,10:00:00,10,0,VEHICLE BREAK-IN/THEFT,...,NONE,0,NaN,NaN,32.668489,-117.090432,VEHICLE BREAK-IN/THEFT,cns/segment,65,POINT (-117.09043 32.66849)
538,2007-01-01 10:00:00,2007,1,1,1,0,10:00:00,10,0,VEHICLE BREAK-IN/THEFT,...,NONE,0,NaN,NaN,33.182076,-117.295949,VEHICLE BREAK-IN/THEFT,cns/segment,65,POINT (-117.29595 33.18208)
535,2007-01-01 10:00:00,2007,1,1,1,0,10:00:00,10,0,VEHICLE BREAK-IN/THEFT,...,NONE,0,NaN,NaN,32.956325,-117.031886,VEHICLE BREAK-IN/THEFT,cns/segment,65,POINT (-117.03189 32.95632)
540,2007-01-01 10:00:00,2007,1,1,1,0,10:00:00,10,0,THEFT/LARCENY,...,NONE,0,NaN,NaN,32.956325,-117.031886,PETTY THEFT,cns/segment,65,POINT (-117.03189 32.95632)
533,2007-01-01 10:00:00,2007,1,1,1,0,10:00:00,10,0,FRAUD,...,NONE,0,NaN,NaN,32.939817,-117.054802,FRAUD,cns/segment,65,POINT (-117.05480 32.93982)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10596,2013-04-30 10:36:00,2013,4,30,18,1,10:36:00,10,0,BURGLARY,...,NONE,0,NaN,NaN,33.066840,-117.303324,RESIDENTIAL BURGLARY,cns/segment,65,POINT (-117.30332 33.06684)
10595,2013-04-30 10:36:00,2013,4,30,18,1,10:36:00,10,0,ASSAULT,...,San002,142456,4.0,1414.0,32.767120,-117.252361,CAUSE GREAT BODILY HARM/DEATH OF ELDER/DEPENDE...,cns/segment,65,POINT (-117.25236 32.76712)
10598,2013-04-30 10:46:00,2013,4,30,18,1,10:46:00,10,0,DRUGS/ALCOHOL VIOLATIONS,...,NONE,0,NaN,NaN,33.058718,-117.217104,USE/UNDER INFL OF CONTROLLED SUBS (M),cns/segment,65,POINT (-117.21710 33.05872)
10597,2013-04-30 10:46:00,2013,4,30,18,1,10:46:00,10,0,ASSAULT,...,NONE,0,NaN,NaN,0.000000,0.000000,BATTERY ON PERSON (M),NONE,-1,POINT (0.00000 0.00000)
